In [ ]:
import spacy.lang.en
import plotly.express as px
from mair import papers_processing_utils

import pandas as pd    
import glob
from tqdm import tqdm
import spacy.lang.en
from collections import defaultdict
from spacy.matcher import PhraseMatcher
import os
from mair import papers_processing_utils
import json
from sklearn.preprocessing import MultiLabelBinarizer

tqdm.pandas()

In [ ]:
keywords = [
    "privacy",
    "fairness",
    "accountability",
    "transparency",
    "safety",
    "cybersecurity",
    "security",
    "sustainability",
    "oversight",
    "solidarity",
    "interpretability",
    "policy",
    "regulation",
    "responsibility",
]

AI_PAPERS_IN_PATH='../data/s2orc/ai_papers.csv'

In [ ]:
en=spacy.lang.en.English()

matcher = PhraseMatcher(en.vocab, attr="NORM") #TODO: change to lemma
for pattern in keywords:
    matcher.add(pattern, None, en(pattern))


In [ ]:
df = pd.read_csv(AI_PAPERS_IN_PATH)

In [ ]:
count = pd.DataFrame(df.groupby('year').count()['paper_id'])
count = count.rename(columns={'paper_id': 'papers_count'})

In [ ]:
count.to_csv('ai_papers_count.csv')

In [ ]:
len(df)

In [ ]:
df['cleaned_abstract'] = df['abstract'].str.replace('\n',' ')

text_to_search = df.apply(lambda row: str(row['title'])+' '+str(row['cleaned_abstract']), axis=1).str.lower()

docs = text_to_search.apply(en)

In [ ]:
matching_results = docs.apply(matcher)

In [ ]:
m = matching_results.apply(lambda matches: list(set([en.vocab.strings[id] for id, _, _ in matches])))

In [ ]:
mlb = MultiLabelBinarizer(classes=keywords)

In [ ]:
a = mlb.fit_transform(m)

keywords_df = pd.DataFrame(a, m.index, mlb.classes_)
keywords_df

In [ ]:
df2 = keywords_df.join(df)

In [ ]:
yearly_keywords = df2.groupby('year')[keywords].sum()

In [ ]:
px.line(yearly_keywords)

In [ ]:
yearly_keywords.to_csv('yearly_keywords.csv')

In [ ]:
keywords_df.corr()